# Dealing with Sparsity via Imputation into Filitered Numerical data from Positive Calss

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import kurtosis, skew
from scipy import stats
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.preprocessing import Imputer

In [4]:
pos = pd.read_table("Stage2_output_num.csv", sep='\t')

In [5]:
# double check columns were removed.
col_indexes_to_remove = [] 
for i in range(1,(pos.shape[1]-1)):
    Count_the_Na = sum(pos.iloc[:,i].isnull())
    #print Count_the_Na
    if Count_the_Na == pos.shape[0]:
        col_indexes_to_remove.append(i)
print col_indexes_to_remove

[]


# Assessing best method of imputation for each column
# Based on distribution of column 

If columns contain an appropriate number of Nas:
- Columns with normal distribution will be imputed with mean
- Columns with high skew are imputed with median - more strigent criteria applied to negative skew as most columns with negatively skewed
- Else apply regression 

</br>
Else imputation would be inappropriate if too many Nas in column - poor imputation could introduce bias.

In [6]:
indexes_to_impute_mean = []
indexes_to_impute_median = []
indexes_for_regression = []
indexes_for_toManyNas = []

for i in range(1,(pos.shape[1]-1)):  # Exclude id and response variables from loop
    column = pos.iloc[:,i].dropna()
    
    shapiro_p_val = stats.shapiro(column)[1]  # if p value is more than 0.05 = could be normal 
    range_descript = column.describe()
    IQ_ratio_R = (range_descript[6] - range_descript[4]) / (range_descript[7] - range_descript[3])
    
    # Ratio of Nas to non Nas
    Na_ratio = pos.iloc[:,i].isnull().sum() / np.uint64(pos.iloc[:,i].shape[0])
    if (Na_ratio < 0.3):
        if (shapiro_p_val > 0.05):
            # if less than 0.05, we cannot reject H0 that sample comes from a normal distribution
            # so if shapiro is more than 0.05, impute mean.
            indexes_to_impute_mean.append(i)
            print "impute mean"
        elif (skew(column) > 1.5): # if positive skew inpute median
            indexes_to_impute_median.append(i)
            print "impute median"
        elif (skew(column) < 1.75) and (kurtosis(column) > 2): # if negative skew with high kurtosis, impute median
            indexes_to_impute_median.append(i)
            print "impute median"
        elif (skew(column) < 2) and (IQ_ratio_R < 0.2):
            indexes_to_impute_median.append(i)
            print "impute median"
        else:
            print "require_regression"
            indexes_for_regression.append(i)
    else: 
        indexes_for_toManyNas.append(i)
        print "Too Sparse for imputation"

Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
Too Sparse for imputation
require_regression
require_regression
require_regression
require_regression
impute median
impute median
impute median
impute median
require_regression
require_regression
require_regression
require_regression
require_regression
require_regression
impute median
require_regression
require_regression
impute median
require_regression
impute median
require_regression
impute median
require_regression
impute median
require_regression
require_regression
require_regression
impute median
impute median
impute median
require_regression
impute median
impute median
i

//anaconda/lib/python2.7/site-packages/scipy/stats/morestats.py:1327: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


In [17]:
print("Indexes for Mean Imputation: %s \n" % (indexes_to_impute_mean))
print("Indexes for Median Imputation: %s \n" % (indexes_to_impute_median))
print("Indexes for Regression Imputation: %s \n" % (indexes_for_regression))
print("Leave Alone: %s \n" % (indexes_for_toManyNas))


Indexes for Mean Imputation: [91, 92, 95, 96, 103, 104, 111, 112, 115, 116, 140, 149, 150, 161, 162] 

Indexes for Median Imputation: [22, 23, 24, 25, 32, 35, 37, 39, 41, 45, 46, 47, 49, 50, 51, 52, 53, 55, 60, 62, 64, 65, 66, 67, 68, 69, 70, 71, 72, 75, 80, 83, 86, 93, 94, 98, 100, 102, 106, 107, 108, 114, 117, 118, 119, 120, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 142, 143, 145, 146, 147, 148, 151, 152, 163, 164] 

Indexes for Regression Imputation: [18, 19, 20, 21, 26, 27, 28, 29, 30, 31, 33, 34, 36, 38, 40, 42, 43, 44, 48, 54, 56, 57, 58, 59, 61, 63, 73, 74, 76, 77, 78, 79, 81, 82, 84, 85, 87, 88, 89, 90, 97, 99, 101, 105, 109, 110, 113, 121, 122, 123, 124, 141, 144] 

Leave Alone: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 153, 154, 155, 156, 157, 158, 159, 160] 



# Imputation of columns with with at most 0.3 ratio between Na vs NonNa
Imputation may help deal with sparsitiy - but it can also introduce bias <br>
Thus we will sample only 75 % of these initial chosen features to impute

In [19]:
import random

In [29]:
indexes_to_impute_mean = random.sample(indexes_to_impute_mean, int(round(len(indexes_to_impute_mean)*0.75))) # no replacement

In [30]:
indexes_to_impute_median = random.sample(indexes_to_impute_median, int(round(len(indexes_to_impute_median)*0.75))) # no replacement

In [31]:
indexes_for_regression = random.sample(indexes_for_regression, int(round(len(indexes_for_regression)*0.75))) # no replacement

# Impute median values for skewed columns with high kurtosis / low ratio between interquartile range and range

In [32]:
for i in indexes_to_impute_median:
    fillIt = pos.iloc[:,i].dropna().median()
    pos.iloc[:,i] = pos.iloc[:,i].fillna(value=fillIt)

# Impute mean values for columns that was unable to reject the null hypothesis of Shapiro-wilks test for normality

In [ ]:
for i in indexes_to_impute_mean:
    fillIt = pos.iloc[:,i].dropna().mean()
    pos.iloc[:,i] = pos.iloc[:,i].fillna(value=fillIt)

# Imputation via regression techniques

# Models used:
## 1- Linear Regression
## 2- Bayesian Ridge
## 3- AdaBoost Regressor 

In [38]:
reg1 = linear_model.LinearRegression()
reg2 = linear_model.BayesianRidge()
reg3 = AdaBoostRegressor()

In [39]:
# setting up feature selection and imputer
selector = RFE(reg1, 10, step=1)
imputer = Imputer()

In [40]:
# Thus function applies a regression model
# Then it calculates and returns SSR from resubtitution to Non-Na values to y (target) 
# Then it returns the predictions for the all values of Y

def RegressionModel(XIn,Xtest, YIn, clfIn, doPCABool=False):
    YYflattened = YIn.values.flatten()
 
    if (doPCABool == True): 
        x = StandardScaler().fit_transform(XIn)
        pca = PCA(n_components=int(round(XIn.shape[1]/2)))
        principalComponents = pca.fit_transform(x) # transform Train set using PCA
        
        clfIn.fit(principalComponents, YYflattened) # predict Y using trainset
        
        Resubstitution_preds = clfIn.predict(principalComponents)  # predict Y using trainset PCA'd
        
        x_pca = StandardScaler().fit_transform(Xtest) 
        imputed_pcad = pca.fit_transform(x_pca) 
        Generalized_preds = clfIn.predict(imputed_pcad)
        
    else:
        clfIn.fit(XIn, YYflattened) # train X train set to predict Y
        Resubstitution_preds = clfIn.predict(XIn) # Predict y using train set
        Generalized_preds = clfIn.predict(Xtest) # predict y using IMPUTED TEST set


    resids =(YYflattened - Resubstitution_preds)**2  # Test resubstution error
    
    return [sum(resids), Generalized_preds]

# Predict each suitable column with model that returned best resubstitution error 
### For each model, attempt model again with PCA input / features selected from recursive selection with linear regression.

In [ ]:
allIndexes = set(range(1,(pos.shape[1]-1))) # all indexes

# all indexes take away union of too many nan and reg indexes
AverageImputedCols = allIndexes - set(indexes_for_regression).union(indexes_for_toManyNas) 

for i in indexes_for_regression:
    Yy = pos.iloc[:,i] # test set
    predictor_Col = {i} # column to predict
    trainIndexes =  AverageImputedCols - predictor_Col # train indexes - indexes other than the predictor column
    trainIndexes = list(trainIndexes) 
    Xx = pos.iloc[:,trainIndexes] # train set with NAS
    xxyy = pd.concat([Xx, Yy], axis=1) # all set
    xxyy_no_na = xxyy.dropna(axis="index") # drop nas by index.
    YY = xxyy_no_na[[Yy.name]] # y set with NO Nas
    XX = xxyy_no_na.loc[:, xxyy_no_na.columns != Yy.name] # train set with NO nas
    
    Xx_imputed = imputer.fit_transform(pd.DataFrame(Xx)) # uses mean/median/mode
    Xx_imputed = pd.DataFrame(Xx_imputed, columns=[Xx.columns])
    
    ### Apply models 
    m1 = RegressionModel(XIn=XX,Xtest=Xx_imputed, YIn=YY, clfIn=reg1) # Linear Regression
    m2 = RegressionModel(XIn=XX,Xtest=Xx_imputed, YIn=YY, clfIn=reg2) # Bayesian Ridge
    m3 = RegressionModel(XIn=XX,Xtest=Xx_imputed, YIn=YY, clfIn=reg3) # AdaBoostRegressor
    m4 = RegressionModel(XIn=XX,Xtest=Xx_imputed, YIn=YY, clfIn=reg1, doPCABool=True) 
    m5 = RegressionModel(XIn=XX,Xtest=Xx_imputed, YIn=YY, clfIn=reg2, doPCABool=True)
    m6 = RegressionModel(XIn=XX,Xtest=Xx_imputed, YIn=YY, clfIn=reg3, doPCABool=True)
    # Feature selection may take too much time
    # so will do one FS - recursive selection with linear regression.
    selector = selector.fit(XX, YY.values.flatten())
    FS_XX = XX[XX.columns[selector.support_]]
    FS_Xx_Imputed = Xx_imputed[Xx_imputed.columns[selector.support_]]
    
    m7 = RegressionModel(XIn=FS_XX,Xtest=FS_Xx_Imputed, YIn=YY, clfIn=reg1) # Linear Regression with FS
    m8 = RegressionModel(XIn=FS_XX,Xtest=FS_Xx_Imputed, YIn=YY, clfIn=reg2) # Bayesian Ridge with FS
    m9 = RegressionModel(XIn=FS_XX,Xtest=FS_Xx_Imputed, YIn=YY, clfIn=reg3) # AdaBoostRegressor with FS
    m10 = RegressionModel(XIn=FS_XX,Xtest=FS_Xx_Imputed, YIn=YY, clfIn=reg1, doPCABool=True)# PC's cut with FS
    m11 = RegressionModel(XIn=FS_XX,Xtest=FS_Xx_Imputed, YIn=YY, clfIn=reg2, doPCABool=True) # rinse, repeat
    m12 = RegressionModel(XIn=FS_XX,Xtest=FS_Xx_Imputed, YIn=YY, clfIn=reg3, doPCABool=True)
    
    ssr_collect = [m1[0], m2[0], m3[0], m4[0], m5[0], m6[0],
                        m7[0], m8[0], m9[0], m10[0], m11[0], m12[0]]
    pred_collect = [m1[1], m2[1], m3[1], m4[1], m5[1], m6[1],
                    m7[1], m8[1], m9[1], m10[1], m11[1], m12[1]]
    
    # find model which produced the smallest sum of square residuals
    sorted_ssr, sorted_pred = zip(*sorted(zip(ssr_collect, pred_collect)))
    best_pred = sorted_pred[0] # best ssr
    
    output =pd.DataFrame(best_pred, columns=["s"])
    for j in range(0,pos.shape[0]):
        pos.iloc[j,i] = output.iloc[j,0]

# Write out partially imputed dataset

In [42]:
pos.to_csv("Stage3_num_partially_IMPUTED.csv", sep='\t', index= False )